In [13]:
%load_ext autoreload
%autoreload 2

In [12]:
import JGGame

board_arr = [  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0, -5,  0,  0,  0, -1,  0,  0,  0, -5,
       18,  0,  0,  0,  0,  0,  0,  0, -1,  0,  0, -1,  0,  0,  0,  0,  0,
        0,  0,  0,  0, -2,  0,  0,  0, -2, -1,  0,  0]

JGGame.flood_fill(board_arr, 0, 34, 10)

[42, 33]

In [7]:
JGGame.ix_to_ax[34]

(4, 0)

In [14]:
def adjacent_idxs(idx: int, player_idx: int):
    q, r = JGGame.ix_to_ax[idx]
    adj_coords = [
        ((q + 1, r), True),
        ((q - 1, r), True),
        ((q, r + 1), player_idx == 1),
        ((q, r - 1), player_idx == 0),
        ((q + 1, r + 1), player_idx == 1),
        ((q - 1, r - 1), player_idx == 0),
    ]
    res = []
    for coord, valid in adj_coords:
        if not valid:
            continue
        idx = JGGame.ax_to_ix.get(coord)
        if idx is None:
            continue
        if idx == JGGame.PLAYER_IDX_CITY_IDXS[player_idx]:
            continue
        res.append(idx)
    return res

def flood_fill(board, player_idx: int, start_idx: int, count: int):
    valid_moves = []
    # Players can't move coins into their own city
    visited = set([JGGame.PLAYER_IDX_CITY_IDXS[player_idx]])
    queue = [(start_idx, count, True)]
    while queue:
        idx, count, is_first = queue.pop()
        if idx in visited:
            print(f"Already visited {idx}")
            continue
        visited.add(idx)

        print(f"Visiting {idx}: {board[idx]}")
        if board[idx] <= 0:
            # If the tile is empty or owned by the opponent, it's a valid move
            valid_moves.append(idx)

        # Continue moving if:
        # - count > 0, and
        # - the current tile is empty
        if count > 0 and (is_first or board[idx] == 0):
            queue.extend(
                (adj_idx, count - 1, False)
                for adj_idx in adjacent_idxs(idx, player_idx)
                if adj_idx not in visited
            )

    return valid_moves

b = JGGame.Board(board_arr)

b.display()

flood_fill(board_arr, 0, 34, 10)

    _ _ _ _ _ 
   _ _ _ _ _ _ 
  _ _ _ _ _ _ _ 
 _ _ _ _ _ _ _ 5 
_ _ _ 1 _ _ _ 5 18 
 _ _ _ _ _ _ _ 1 
  _ _ 1 _ _ _ _ 
   _ _ _ _ _ 2 
    _ _ _ 2 1 
Player 1: 0
Player 2: 0
Visiting 34: 18
Visiting 42: -1
Visiting 33: -5


[42, 33]

In [8]:
from utils import dotdict

args = dotdict({
    'numIters': 1000,
    'numEps': 50,              # Number of complete self-play games to simulate during a new iteration.
    #'numIters': 1000,
    #'numEps': 1,               # Number of complete self-play games to simulate during a new iteration.
    'tempThreshold': 15,        #
    'updateThreshold': 0.6,     # During arena playoff, new neural net will be accepted if threshold or more of games are won.
    'maxlenOfQueue': 200000,    # Number of game examples to train the neural networks.
    #'numMCTSSims': 25,          # Number of games moves for MCTS to simulate.
    'numMCTSSims': 25,          # Number of games moves for MCTS to simulate.
    'arenaCompare':  20,        # Number of games to play during arena play to determine if new net will be accepted.
    'cpuct': 1,

    'checkpoint': f'./checkpoints-JGGame/',
    'load_model': False,
    'load_folder_file': 'best.pth.tar',
    'numItersForTrainExamplesHistory': 20,

})

In [10]:
import JGGame
from JGNet import NNetWrapper
from MCTS import MCTS

game = JGGame.JGGame()

x = NNetWrapper(game)
x.load_checkpoint('./checkpoints-JGGame/best.pth.tar')
m = MCTS(game, x, args)

In [15]:
import numpy as np


board_arr = [  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0, -5,  0,  0,  0, -1,  0,  0,  0, -5,
       15,  0,  0,  0,  0,  0,  0,  0, -1,  0,  0, -1,  0,  0,  0,  0,  0,
        0,  0,  0,  0, -2,  0,  0,  0, -2, -1,  0,  0]
m.getActionProb(np.array(board_arr))

AssertionError: count: 18

In [14]:
import JGGame

JGGame.PLAYER_STARTING_IDXS

for src_idx_player in range(0b1111 + 1):
    for dst_idx in range(0b111100 + 1):
        for count in range(1, 0b1111 + 1):
            packed = JGGame.action_pack(False, src_idx_player, dst_idx, count)
            unpacked = JGGame.action_unpack(packed)
            assert not unpacked[0]
            assert unpacked[1] == src_idx_player
            assert unpacked[2] == dst_idx
            assert unpacked[3] == count

JGGame.action_unpack(JGGame.action_pack(True, 0, 10, 3))

(True, 0, 0, 0)

In [17]:
game = JGGame.JGGame()

board = game.getInitBoard()
game.getValidMoves(board, 1)

array([False,  True,  True, ..., False, False, False])

In [24]:
game = JGGame.JGGame()

board = game.getInitBoard()
board = game.getNextState(board, 1, JGGame.action_pack(False, 0, 10, 3))[0]
board[:15]

array([[0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 3],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0]], dtype=int8)